<a href="https://colab.research.google.com/github/KorStats/ocr_receipt/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [28]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.11.6
    Uninstalling tokenizers-0.11.6:
      Successfully uninstalled tokenizers-0.11.6
  Attempting uninstall: transformers
    Found existing installation: transformers 4.17.0
    Uninstalling transformers-4.17.0:
      Successfully uninstalled transformers-4.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kobert 0.2.3 requires transformers>=4.8.1, but you have transformers 3.0.2 which is incompatible.
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-frzqrrv_
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-frzqrrv_
  Using cached transformers-4

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 산업분류자동화  'Colab Notebooks'  'My Drive'	'실습용 자료.zip의 사본'


In [31]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'모델 개발용 자료'.zip -d /content/dataset
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/모델 개발용 자료.zip
2. 모델개발용자료.txt:  mismatching "local" filename (2. ыкиыН╕ъ░Ьы░ЬьЪйьЮРыгМ.txt),
         continuing with "central" filename version
replace /content/dataset/2. 모델개발용자료.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/dataset/2. 모델개발용자료.txt  
Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
replace /content/dataset/1. 실습용자료.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/dataset/1. 실습용자료.txt  


In [82]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")
ts=pd.read_csv('/content/dataset/2. 모델개발용자료.txt', sep = "|", encoding = "euc-kr")

In [83]:
df.head(3)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매


# 데이터 전처리

In [84]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)

i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)

df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제

df.head(10)

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000001,S,95,952,카센터에서 자동차부분정비 타이어오일교환
1,id_0000002,G,47,472,상점내에서 일반인을 대상으로 채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,id_0000004,G,47,475,영업점에서 일반소비자에게 열쇠잠금장치
4,id_0000005,Q,87,872,어린이집 보호자의 위탁을 받아 취학전아동보육
5,id_0000006,C,29,291,철 절삭.용접 카프라배관자재
6,id_0000007,I,56,561,음식점에서 접객시설을 갖추고 참치회(일본식)
7,id_0000008,C,10,107,쌀을 가지고 가공하여 떡제조
8,id_0000009,O,84,841,시청에서 재정과인력 일반공공행정 지방행정 집행
9,id_0000010,F,42,423,영업장에서 고객의뢰를 받아 내부전기공사


In [85]:
df=df[:100]   # 데이터 너무 많아서 런타임 끊길까봐

# 띄어쓰기 교정하기

In [86]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-isqj0j1z
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-isqj0j1z


In [87]:
from hanspell import spell_checker 

In [88]:
df.shape

(100, 5)

In [89]:
temp_result=[]
for sentence in df['text']:
  result=spell_checker.check(sentence)
  temp_result.append(result.checked)

df['text']=pd.DataFrame(temp_result)      #100개 문장 검사하는데 6초걸림

In [90]:
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000001,S,95,952,카센터에서 자동차 부분정비 타이어 오일 교환
1,id_0000002,G,47,472,상점 내에서 일반인을 대상으로 채소. 과일 판매
2,id_0000003,G,46,467,절단하여 사업체에도 매 공업용 고무를 가지고 합성고무도 매
3,id_0000004,G,47,475,영업점에서 일반 소비자에게 열쇠 잠금장치
4,id_0000005,Q,87,872,어린이집 보호자의 위탁을 받아 취학 전 아동보육
5,id_0000006,C,29,291,철 절삭. 용접 카프라 배관자재
6,id_0000007,I,56,561,음식점에서 접객시설을 갖추고 참치 회(일본식)
7,id_0000008,C,10,107,쌀을 가지고 가공하여 떡 제조
8,id_0000009,O,84,841,시청에서 재정과 인력 일반 공공행정 지방행정 집행
9,id_0000010,F,42,423,영업장에서 고객 의뢰를 받아 내부 전기공사


# 데이터 증분 코드

In [91]:
import random
import pickle
import re

#wordnet = {}
#with open("wordnet.pickle", "rb") as f:
#	wordnet = pickle.load(f)


In [92]:

# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	
	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)



def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))

	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))

	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))

	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

In [93]:
df=df[:5] #더 줄임...

temp_result=[]
n=0
for sentence in df['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=4)
  id_list=['p'+str(df['AI_id'][n]), 'p'+str(df['AI_id'][n]), 'p'+str(df['AI_id'][n]), 'p'+str(df['AI_id'][n])]
  digit_1_list=[df['digit_1'][n], df['digit_1'][n], df['digit_1'][n], df['digit_1'][n]]
  digit_2_list=[df['digit_2'][n], df['digit_2'][n], df['digit_2'][n], df['digit_2'][n]]
  digit_3_list=[df['digit_3'][n], df['digit_3'][n], df['digit_3'][n], df['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list[1:]}
  df_to_insert = pd.DataFrame(data_to_insert)
  df= df.append(df_to_insert, ignore_index=True)
  n=n+1


In [94]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000001,S,95,952,카센터에서 자동차 부분정비 타이어 오일 교환
1,id_0000002,G,47,472,상점 내에서 일반인을 대상으로 채소. 과일 판매
2,id_0000003,G,46,467,절단하여 사업체에도 매 공업용 고무를 가지고 합성고무도 매
3,id_0000004,G,47,475,영업점에서 일반 소비자에게 열쇠 잠금장치
4,id_0000005,Q,87,872,어린이집 보호자의 위탁을 받아 취학 전 아동보육
5,pid_0000001,S,95,952,카센터에서 자동차 부분정비 타이어 오일 교환
6,pid_0000001,S,95,952,카센터에서 자동차 부분정비 타이어 오일 교환
7,pid_0000001,S,95,952,카센터에서 자동차 부분정비 타이어 오일 교환
8,pid_0000001,S,95,952,카센터에서 자동차 부분정비 타이어 오일 교환
9,pid_0000002,G,47,472,상점 채소. 일반인을 대상으로 내에서 과일 판매


## 데이터 전처리

In [ ]:
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육
5,id_0000006,C,29,291,철,절삭.용접,카프라배관자재
6,id_0000007,I,56,561,음식점에서,접객시설을 갖추고,참치회(일본식)
7,id_0000008,C,10,107,쌀을 가지고,가공하여,떡제조
8,id_0000009,O,84,841,시청에서 재정과인력,일반공공행정,지방행정 집행
9,id_0000010,F,42,423,영업장에서,고객의뢰를 받아,내부전기공사


In [ ]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000001,S,95,952,카센터에서 자동차부분정비 타이어오일교환
1,id_0000002,G,47,472,상점내에서 일반인을 대상으로 채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,id_0000004,G,47,475,영업점에서 일반소비자에게 열쇠잠금장치
4,id_0000005,Q,87,872,어린이집 보호자의 위탁을 받아 취학전아동보육
5,id_0000006,C,29,291,철 절삭.용접 카프라배관자재
6,id_0000007,I,56,561,음식점에서 접객시설을 갖추고 참치회(일본식)
7,id_0000008,C,10,107,쌀을 가지고 가공하여 떡제조
8,id_0000009,O,84,841,시청에서 재정과인력 일반공공행정 지방행정 집행
9,id_0000010,F,42,423,영업장에서 고객의뢰를 받아 내부전기공사


In [ ]:
# 결측값 확인
df.isnull().sum()

AI_id      0
digit_1    0
digit_2    0
digit_3    0
text       0
dtype: int64

In [ ]:
df.shape

(1000000, 5)

In [ ]:
df['digit_1'].value_counts()

G    246472
I    187425
C    105192
S    100396
H     98038
P     46610
L     40140
Q     36087
F     35050
R     29751
M     28434
N     17701
J     10862
K     10378
O      2965
E      2255
A      1064
D       756
B       424
Name: digit_1, dtype: int64

In [ ]:
df_0=df[df['digit_1']=='G']
print(df_0[['digit_2']].value_counts())
print(df_0[['digit_3']].value_counts())

digit_2
47         162202
46          76835
45           7435
dtype: int64
digit_3
474        36453
478        30566
471        26189
472        25668
464        18453
463        17873
465        14652
475        13031
467        12096
473         9312
479         9218
466         8154
476         7188
452         4696
477         4577
462         2585
461         2328
451         2268
468          694
453          471
dtype: int64


In [ ]:
df_1=df[df['digit_1']=='I']
print(df_1[['digit_2']].value_counts())
print(df_1[['digit_3']].value_counts())

digit_2
56         173459
55          13966
dtype: int64
digit_3
561        123703
562         49756
551         12549
559          1417
dtype: int64


In [ ]:
df_2=df[df['digit_1']=='C']
print(df_2[['digit_2']].value_counts())
print(df_2[['digit_3']].value_counts())

digit_2
25         17065
10         14576
29         12087
22          5738
28          5524
14          5417
13          5287
33          4584
18          4177
34          3380
26          3306
32          3275
27          3207
20          3029
23          2873
30          2772
24          2215
17          1754
16          1724
15          1379
31          1032
11           387
21           313
19            88
12             3
dtype: int64
digit_3
259        12876
107         9214
292         7163
222         4935
291         4924
           ...  
304           22
301           20
191           20
266            8
120            3
Length: 85, dtype: int64


In [ ]:
df_3=df[df['digit_1']=='S']
print(df_3[['digit_2']].value_counts())
print(df_3[['digit_3']].value_counts())

digit_2
96         56210
94         23625
95         20561
dtype: int64
digit_3
961        43040
949        21580
969        13170
952        12142
953         6911
941         1612
951         1508
942          433
dtype: int64


In [ ]:
df_4=df[df['digit_1']=='S']
print(df_4[['digit_2']].value_counts())
print(df_4[['digit_3']].value_counts())

digit_2
96         56210
94         23625
95         20561
dtype: int64
digit_3
961        43040
949        21580
969        13170
952        12142
953         6911
941         1612
951         1508
942          433
dtype: int64


In [ ]:
df_0.head(3)

,AI_id,digit_1,digit_2,digit_3,text
1,id_0000002,G,47,472,상점내에서 일반인을 대상으로 채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,id_0000004,G,47,475,영업점에서 일반소비자에게 열쇠잠금장치


In [ ]:
folder_name = './data'
file_name = 'klue_extra_data.csv'
file_path = os.path.join(folder_name, file_name)

NameError: ignored

In [ ]:
klue_off_dev = pd.read_csv(file_path)

In [ ]:
from pororo import Pororo
mt = Pororo(task="translation", lang="multi")

In [ ]:
def back_trans_pororo(original_text, lang='en'):
    text_to_lang = mt(original_text, src="ko", tgt=lang)
    new_text = mt(text_to_lang, src=lang, tgt="ko")
    return new_text

In [ ]:
lang = 'en'
klue_off_dev_pororo_en_premise = klue_off_dev.copy()
klue_off_dev_pororo_en_premise['premise'] = klue_off_dev['premise'].progress_apply(lambda x: back_trans_pororo(x, lang=lang))

In [ ]:
folder_name = './data'
file_name = 'klue_extra_data_pororo_en_premise.csv'
file_path = os.path.join(folder_name, file_name)
print(f"file_path : {file_path}")
klue_off_dev_pororo_en_premise.to_csv(file_path,index=False)

In [ ]:
klue_off_dev_pororo_en_premise = pd.read_csv(file_path)
klue_off_dev_pororo_en_premise

# 테스트 간단히

In [ ]:
test=pd.read_csv('test.csv')
submission=pd.read_csv('submission.csv')

label_answer=[]
step=0
for i in tqdm(test['title']):
  label_answer.append(predict(i))
  

In [ ]:
df['digit_1']=='G']['digit_2'].value_counts()

KeyError: ignored



#### **추후 과제: 분류가 현재 매우 불균형함 / 정규화를 이용해서 균형을 맞추면 정확도가 올라가는지 확인 필요**


In [ ]:
df.loc[(df['digit_1'] == "G"), 'digit_1'] = 0  
df.loc[(df['digit_1'] == "I"), 'digit_1'] = 1  
df.loc[(df['digit_1'] == "C"), 'digit_1'] = 2  
df.loc[(df['digit_1'] == "S"), 'digit_1'] = 3  
df.loc[(df['digit_1'] == "H"), 'digit_1'] = 4  
df.loc[(df['digit_1'] == "P"), 'digit_1'] = 5  
df.loc[(df['digit_1'] == "L"), 'digit_1'] = 6  
df.loc[(df['digit_1'] == "Q"), 'digit_1'] = 7  
df.loc[(df['digit_1'] == "F"), 'digit_1'] = 8  
df.loc[(df['digit_1'] == "R"), 'digit_1'] = 9  
df.loc[(df['digit_1'] == "M"), 'digit_1'] = 10  
df.loc[(df['digit_1'] == "N"), 'digit_1'] = 11 
df.loc[(df['digit_1'] == "J"), 'digit_1'] = 12  
df.loc[(df['digit_1'] == "K"), 'digit_1'] = 13  
df.loc[(df['digit_1'] == "O"), 'digit_1'] = 14  
df.loc[(df['digit_1'] == "E"), 'digit_1'] = 15  
df.loc[(df['digit_1'] == "A"), 'digit_1'] = 16 
df.loc[(df['digit_1'] == "D"), 'digit_1'] = 17  
df.loc[(df['digit_1'] == "B"), 'digit_1'] = 18  

In [ ]:
data_list = []
for q, label in zip(df['text'], df['digit_1'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['카센터에서 자동차부분정비 타이어오일교환', '3'], ['상점내에서 일반인을 대상으로 채소.과일판매', '0'], ['절단하여사업체에도매 공업용고무를가지고 합성고무도매', '0'], ['영업점에서 일반소비자에게 열쇠잠금장치', '0'], ['어린이집 보호자의 위탁을 받아 취학전아동보육', '7']]


## Train data & test data

In [ ]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
data_train[0]

(array([   2, 1998, 6903,  994, 5816, 4799, 6141,    3,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(8, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=19,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/14063 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.9370217323303223 train acc 0.015625
epoch 1 batch id 201 loss 2.758655548095703 train acc 0.10478855721393035
epoch 1 batch id 401 loss 2.4969706535339355 train acc 0.16696539900249377


# 문장 test 해보기

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)  
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
        return np.argmax(logits)

            if np.argmax(logits) == 0:
                test_eval.append('G 도매 및 소매업(45~47)')
            elif np.argmax(logits) == 1:
                test_eval.append('I 숙박 및 음식점업(55~56)')
            elif np.argmax(logits) == 2:
                test_eval.append('C 제조업(10~34)')
            elif np.argmax(logits) == 3:
                test_eval.append('S 협회 및 단체, 수리 및 기타 개인 서비스업(94~96)')
            elif np.argmax(logits) == 4:
                test_eval.append('H 운수 및 창고업(49~52)')
            elif np.argmax(logits) == 5:
                test_eval.append('P 교육 서비스업(85)')
            elif np.argmax(logits) == 6:
                test_eval.append('L 부동산업(68)')
            elif np.argmax(logits) == 7:
                test_eval.append('Q 보건업 및 사회복지 서비스업(86~87)')
            elif np.argmax(logits) == 8:
                test_eval.append('F 건설업(41~42)')
            elif np.argmax(logits) == 9:
                test_eval.append('R 예술, 스포츠 및 여가관련 서비스업(90~91)')
            elif np.argmax(logits) == 10:
                test_eval.append('M 전문, 과학 및 기술 서비스업(70~73)')
            elif np.argmax(logits) == 11:
                test_eval.append('N 사업시설 관리, 사업 지원 및 임대 서비스업(74~76)')
            elif np.argmax(logits) == 12:
                test_eval.append('J 정보통신업(58~63)')
            elif np.argmax(logits) == 13:
                test_eval.append('K 금융 및 보험업(64~66)')
            elif np.argmax(logits) == 14:
                test_eval.append('O 공공 행정, 국방 및 사회보장 행정(84)')                               
            elif np.argmax(logits) == 15:
                test_eval.append('E 수도, 하수 및 폐기물 처리, 원료 재생업(36~39)')
            elif np.argmax(logits) == 16:
                test_eval.append('A 농업, 임업 및 어업(01~03)')
            elif np.argmax(logits) == 17:
                test_eval.append('D 전기, 가스, 증기 및 공기 조절 공급업(35)')
            elif np.argmax(logits) == 18:
                test_eval.append('B 광업(05~08)')

# 제출하기

In [ ]:
#ts=ts[0:100]

#ts.head(10)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
5,id_000006,NaN,NaN,NaN,"철,아크릴,포맥스",스크린인쇄,명판
6,id_000007,NaN,NaN,NaN,음식점,접객시설가지고,조개구이판매
7,id_000008,NaN,NaN,NaN,스테인레스를,프레스가공하여제조,주방용품
8,id_000009,NaN,NaN,NaN,수리,서비스센터에서,전문수리 수입차
9,id_000010,NaN,NaN,NaN,"약품(화공), 미싱","완성품입고, 수선",NaN


In [ ]:
ts.shape

In [ ]:
ts['text']=ts['text_obj'].map(str)+" "+ts['text_mthd'].map(str)+" "+ts['text_deal'].map(str)

i=[]
for sentence in ts['text']:
    a=sentence.replace('nan', '')
    i.append(a)
ts['text']=pd.DataFrame(i)

ts=ts.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제

ts.head(10)

,AI_id,digit_1,digit_2,digit_3,text
0,id_000001,NaN,NaN,NaN,치킨전문점에서 고객의주문에의해 치킨판매
1,id_000002,NaN,NaN,NaN,산업공구 다른 소매업자에게 철물 수공구
2,id_000003,NaN,NaN,NaN,절에서 신도을 대상으로 불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서 고객요구로 자동차튜닝
4,id_000005,NaN,NaN,NaN,"실내포장마차에서 접객시설을 갖추고 소주,맥주제공"
5,id_000006,NaN,NaN,NaN,"철,아크릴,포맥스 스크린인쇄 명판"
6,id_000007,NaN,NaN,NaN,음식점 접객시설가지고 조개구이판매
7,id_000008,NaN,NaN,NaN,스테인레스를 프레스가공하여제조 주방용품
8,id_000009,NaN,NaN,NaN,수리 서비스센터에서 전문수리 수입차
9,id_000010,NaN,NaN,NaN,"약품(화공), 미싱 완성품입고, 수선"


In [ ]:
result=[]
for sentence in ts['text']:
  temp=predict(sentence)
  result.append(temp)

ts['digit_1']=pd.DataFrame(result)

ts.to_csv('result.csv', index=False, encoding='utf-8-sig')
ts.head(10)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


,AI_id,digit_1,digit_2,digit_3,text
0,id_000001,I 숙박 및 음식점업(55~56),NaN,NaN,치킨전문점에서 고객의주문에의해 치킨판매
1,id_000002,G 도매 및 소매업(45~47),NaN,NaN,산업공구 다른 소매업자에게 철물 수공구
2,id_000003,"S 협회 및 단체, 수리 및 기타 개인 서비스업(94~96)",NaN,NaN,절에서 신도을 대상으로 불교단체운영
3,id_000004,"S 협회 및 단체, 수리 및 기타 개인 서비스업(94~96)",NaN,NaN,영업장에서 고객요구로 자동차튜닝
4,id_000005,I 숙박 및 음식점업(55~56),NaN,NaN,"실내포장마차에서 접객시설을 갖추고 소주,맥주제공"
5,id_000006,C 제조업(10~34),NaN,NaN,"철,아크릴,포맥스 스크린인쇄 명판"
6,id_000007,I 숙박 및 음식점업(55~56),NaN,NaN,음식점 접객시설가지고 조개구이판매
7,id_000008,C 제조업(10~34),NaN,NaN,스테인레스를 프레스가공하여제조 주방용품
8,id_000009,"S 협회 및 단체, 수리 및 기타 개인 서비스업(94~96)",NaN,NaN,수리 서비스센터에서 전문수리 수입차
9,id_000010,C 제조업(10~34),NaN,NaN,"약품(화공), 미싱 완성품입고, 수선"
